# Scraping and storing all pdfs related to "Eidgenössische Steuerverwaltung"

In [1]:
import urllib
import requests
from bs4 import BeautifulSoup
from re import compile, sub, match, findall

In [2]:
url = "https://www.admin.ch/opc/de/federal-gazette/2015/index_32.html"

On this website you can find pdfs of the The Federal Gazette, which is a publication of the Swiss government. The pdfs related to "Eidgenössische Steuerverwaltung" and "Amtshilfe" contain names of foreign people, which are asked to get in touch immediately with the Swiss government. It is probably a matter of tax fraud.


In [6]:
#first try without range

pdflinks = []

url = "https://www.admin.ch/opc/de/federal-gazette/2015/index_30"
page = BeautifulSoup(requests.get(url))
td = page.find("td", attrs = {"class":"nowrap"})
if td:
    link = td.find("a")
    if link:
        filelink = "http://www.admin.ch"+urllib.unquote(link.get("href"))#.replace("&amp;","&")
        print filelink
        filename = link.text

        urllib.urlretrieve(filelink,"steuer/"+filename+".pdf")

        pdflinks.append(filelink) 


TypeError: object of type 'Response' has no len()

In [10]:
pdflinks = []

url = "https://www.admin.ch/opc/de/federal-gazette/2015/index_32"
page = BeautifulSoup(requests.get(url,headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:5.0)","Referer":"http://google.com"}).text )

for year in range(2011,2016):
    for week in range(1,53):
        url = "https://www.admin.ch/opc/de/federal-gazette/"+str(year)+"/index_"+str(week)
        page = BeautifulSoup(requests.get(url,headers={}).text )

        table = page.find("table","table table-striped")
        if table:
            trs = table.find_all("tr")
            for row in trs:
                doclink = row.find("td", attrs = {"class":"nowrap"})
                if doclink:
                    title = doclink.find_next_sibling("td")
                    if title:
                        if "estv" in title.getText().lower() and "amtshilfe" in title.getText().lower():
                            link = doclink.find("a")
                            filelink = "http://www.admin.ch"+urllib.unquote(link.get("href")).replace("&amp;","&")
                            filename = link.text
                            print filelink

                            #urllib.urlretrieve(filelink,"steuer/"+filename+".pdf") BLOCKED by the Swiss gov

                            pdflinks.append(filelink)
                            
                            time.sleep(2)

http://www.admin.ch/opc/de/federal-gazette/2011/8765.pdf
http://www.admin.ch/opc/de/federal-gazette/2012/8005.pdf
http://www.admin.ch/opc/de/federal-gazette/2012/8032.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/4850.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/4857.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/8399.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/1661.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/1792.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/2022.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/2363.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/3307.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/3748.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/3750.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/3751.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/4132.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/4133.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/5234.pdf
http://www.admin.ch/opc/de/fede

KeyboardInterrupt: 

# New try with WGET

In [ ]:
# ! brew install wget

In [12]:
! wget -r -A.pdf https://www.admin.ch/opc/de/federal-gazette/2015/6091.pdf
# downloading one pdf works

--2015-08-22 16:13:15--  https://www.admin.ch/opc/de/federal-gazette/2015/6091.pdf
Resolving www.admin.ch... 162.23.128.199
Connecting to www.admin.ch|162.23.128.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87914 (86K) [application/pdf]
Saving to: 'www.admin.ch/opc/de/federal-gazette/2015/6091.pdf'

www.admin.ch/opc/de 100%[=====================>]  85.85K  --.-KB/s   in 0.1s   

2015-08-22 16:13:16 (778 KB/s) - 'www.admin.ch/opc/de/federal-gazette/2015/6091.pdf' saved [87914/87914]

FINISHED --2015-08-22 16:13:16--
Total wall clock time: 1.0s
Downloaded: 1 files, 86K in 0.1s (778 KB/s)


In [ ]:
for pdflink in pdflinks:
    !wget -nc --random-wait -P steuer {pdflink}

In [4]:
# Next step: parsing > pdf to text